In [48]:
from fileinput import filename
from importlib.resources import path
from multiprocessing.sharedctypes import Value
import pandas as pd
import os
import openpyxl as ox
from pandas.io.excel import ExcelWriter
from openpyxl import Workbook
import numpy as np
import time
import datetime
start = time.time()

In [49]:
cols = [i for i in range(0,19)] +[i for i in range(25,32)]+[i for i in range(41,72)]
print (cols)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 25, 26, 27, 28, 29, 30, 31, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71]


In [50]:
df = pd.read_excel("d:\ход работ\ГДДО22 СВОД.xlsb", sheet_name='ПЛАН', usecols=cols, header=4) #index_col=False,
#astype('string')

In [51]:
print (df.info)
print (df.dtypes)
isp=df['col31'].unique().tolist()
print(isp)

<bound method DataFrame.info of                  col0                        col1  \
0      97575/12/61643  24/1948/20/ГПД/115-ДО/2020   
1       97575/5/61221  24/1948/20/ГПД/115-ДО/2020   
2      97575/12/61869  24/1948/20/ГПД/115-ДО/2020   
3      97575/14/61426  24/1948/20/ГПД/115-ДО/2020   
4      97575/14/61429  24/1948/20/ГПД/115-ДО/2020   
...               ...                         ...   
23266   97453/2/11081                         NaN   
23267   97453/2/11082                         NaN   
23268   97453/2/11083                         NaN   
23269   97453/3/11528                         NaN   
23270   97453/2/11084                         NaN   

                                 col2                  col3  \
0      ООО "Газпром добыча Астрахань"                   ГПУ   
1      ООО "Газпром добыча Астрахань"                   ГПУ   
2      ООО "Газпром добыча Астрахань"                   ГПУ   
3      ООО "Газпром добыча Астрахань"                   ГПУ   
4      ООО "Газп

In [52]:
'''ОЧИСТКА df 
pd.to_datetime
pd.to_numeric

col_date = [11, 12, 50, 51, 52, 53, 60, 61, 66, 69]
for i in col_date:
    print('col'+str(i))
    df["col"+str(i)] = df["col"+str(i)].astype('string')

col_date = [11, 12, 50, 51, 52, 53, 60, 61, 66, 69]
for i in col_date:
    df[str('col'+str(i))]= pd.to_datetime(df[str('col'+str(i))], errors="ignore", dayfirst=True, format='%d%m%Y',)
    #df[str('col'+str(i))]= pd.to_datetime(df[str('col'+str(i))], dayfirst=True) #, format='%d%m%Y',)
print (df['col11'])
'''

'ОЧИСТКА df \npd.to_datetime\npd.to_numeric\n\ncol_date = [11, 12, 50, 51, 52, 53, 60, 61, 66, 69]\nfor i in col_date:\n    print(\'col\'+str(i))\n    df["col"+str(i)] = df["col"+str(i)].astype(\'string\')\n\ncol_date = [11, 12, 50, 51, 52, 53, 60, 61, 66, 69]\nfor i in col_date:\n    df[str(\'col\'+str(i))]= pd.to_datetime(df[str(\'col\'+str(i))], errors="ignore", dayfirst=True, format=\'%d%m%Y\',)\n    #df[str(\'col\'+str(i))]= pd.to_datetime(df[str(\'col\'+str(i))], dayfirst=True) #, format=\'%d%m%Y\',)\nprint (df[\'col11\'])\n'

In [53]:
#print( df[df['col2']=='ООО "Газпром трансгаз Самара"']['col11'].head)
def replace_date():  # замена строкой
    #df['col11'] = df['col11'].str.replace('г\.', "").str.replace("\.\.", ".").str.replace("*", "")
    col_date = [11, 12, 50, 51, 52, 53, 60, 61, 66, 69]
    for n in col_date:
        df[str('col'+str(n))] = df[str('col'+str(n))].astype('string')
        df[str('col'+str(n))] = df[str('col'+str(n))].str.replace('г\.', "").str.replace("\.\.", ".").str.replace("*", "")       
        #df[str('col'+str(n))]= pd.to_datetime(df[str('col'+str(n))], infer_datetime_format=True, errors="ignore", dayfirst=True, format='%d%m%Y')
        #df[str('col'+str(n))]= pd.to_datetime(df[str('col'+str(n))], infer_datetime_format=True, dayfirst=True, format='%d%m%Y')
        
        
    print( df[df['col2']=='ООО "Газпром добыча Надым"']['col11'].head)

#replace_date()
#print (df.dtypes)

In [54]:
#print( df[df['col2']=='ООО "Газпром трансгаз Самара"']['col11'].head)
def replace_date():  # замена с листа "Замена"
    #df['col11'] = df['col11'].str.replace('г\.', "").str.replace("\.\.", ".").str.replace("*", "")
    path_main ="d:\ход работ\ГДДО22 СВОД.xlsb"
    replacements = pd.read_excel(path_main, sheet_name='Замена') #, index_col=False, header=1)
    #print(replacements)
    rng = len(replacements)
    col_date = [11, 12, 50, 51, 52, 53, 60, 61, 66, 69]

    for n in col_date:
        for i in range(rng):
            df[str('col'+str(n))] = df[str('col'+str(n))].astype('string')
            #df[str('col'+str(n))] = df[str('col'+str(n))].str.replace(replacements['Найти'][i], replacements['Заменить'][i])
            df[str('col'+str(n))] = df[str('col'+str(n))].str.replace('г\.', "").str.replace("\.\.", ".").str.replace("*", "")
            

        #df[str('col'+str(n))] = df[str('col'+str(n))].astype('int')
        df[str('col'+str(n))]= pd.to_datetime(df[str('col'+str(n))], infer_datetime_format=True, errors="ignore", dayfirst=True, format='%d%m%Y')
        
        
    print( df[df['col2']=='ООО "Газпром добыча Надым"']['col11'].head)

#replace_date()
#print (df.dtypes)

In [55]:
pathfile = 'd:\ход работ'
ITC = ['АО "Газпром диагностика" ИТЦ "Ростов" участок Краснодар',
    'АО "Газпром диагностика" ИТЦ "Ростов"',\
    'АО "Газпром диагностика" ИТЦ "Санкт-Петербург"',\
    'АО "Газпром диагностика" ИТЦ "Санкт-Петербург" участок Приобье',\
    'АО "Газпром диагностика" ИТЦ "Санкт-Петербург" участок Томск',\
    'АО "Газпром диагностика" ИТЦ "Видное" (скважины)',\
    'АО "Газпром диагностика" ИТЦ "Видное" (отбраковка)',\
    'АО "Газпром диагностика" ИТЦ "Видное"',\
    'АО "Газпром диагностика" ИТЦ "Саратов"' ]

ITC_ROS = ['АО "Газпром диагностика" ИТЦ "Ростов" участок Краснодар', 'АО "Газпром диагностика" ИТЦ "Ростов"']
ITC_SAN = ['АО "Газпром диагностика" ИТЦ "Санкт-Петербург"', 'АО "Газпром диагностика" ИТЦ "Санкт-Петербург" участок Приобье', 'АО "Газпром диагностика" ИТЦ "Санкт-Петербург" участок Томск']
ITC_VID = ['АО "Газпром диагностика" ИТЦ "Видное" (отбраковка)','АО "Газпром диагностика" ИТЦ "Видное"', 'АО "Газпром диагностика" ИТЦ "Саратов"']
print(ITC)
print(ITC_ROS)
print(ITC_SAN)
print(ITC_VID)

['АО "Газпром диагностика" ИТЦ "Ростов" участок Краснодар', 'АО "Газпром диагностика" ИТЦ "Ростов"', 'АО "Газпром диагностика" ИТЦ "Санкт-Петербург"', 'АО "Газпром диагностика" ИТЦ "Санкт-Петербург" участок Приобье', 'АО "Газпром диагностика" ИТЦ "Санкт-Петербург" участок Томск', 'АО "Газпром диагностика" ИТЦ "Видное" (скважины)', 'АО "Газпром диагностика" ИТЦ "Видное" (отбраковка)', 'АО "Газпром диагностика" ИТЦ "Видное"', 'АО "Газпром диагностика" ИТЦ "Саратов"']
['АО "Газпром диагностика" ИТЦ "Ростов" участок Краснодар', 'АО "Газпром диагностика" ИТЦ "Ростов"']
['АО "Газпром диагностика" ИТЦ "Санкт-Петербург"', 'АО "Газпром диагностика" ИТЦ "Санкт-Петербург" участок Приобье', 'АО "Газпром диагностика" ИТЦ "Санкт-Петербург" участок Томск']
['АО "Газпром диагностика" ИТЦ "Видное" (отбраковка)', 'АО "Газпром диагностика" ИТЦ "Видное"', 'АО "Газпром диагностика" ИТЦ "Саратов"']


In [56]:

''''# update_spreadsheet - функция для записи в книгу на существующий лист с ранее уст форматированием 
В видео несколько вариантов для перезаписи. Если использовать openpyxl напрямую, т.е. по схеме:
wb = ox.load_workbook(path)
wb[sheet_name].cell(startrow + ir, starcol + ic).value = _df.iloc[ir][ic]
то да, формат сохраняется.
'''
def update_spreadsheet(path : str, _df, starcol : int = 0, startrow : int = 5, sheetname : str = "План"):
    wb = ox.load_workbook(path)  # открываем файл
    for ir in range (0, len(_df)):
        for ic in range (0, len(_df.iloc[ir])):
            wb[sheetname].cell(startrow + ir, starcol + ic).value = _df.iloc[ir][ic]  # записываю данные в строки таблицы
    wb.save(path) 

In [57]:
#for i in ITC_ROS:
#print(df['col31'])
#print(df[df['col31']=='АО "Газпром диагностика" ИТЦ "Санкт-Петербург"'])

df1 = df[df['col31'].isin(ITC_ROS)]

print(df1)

xlsx_path = os.path.join(pathfile,"isp", "ГДДО22 СВОД_АО Газпром диагностика ИТЦ Ростов.xlsx") 
update_spreadsheet(xlsx_path, df1.sample(len(df1)), sheetname="План", starcol=1, startrow=6)


                 col0                      col1  \
885    97436/12/02009  24/1966/20/ГПД/130-ДО/20   
886    97436/12/02010  24/1966/20/ГПД/130-ДО/20   
887    97436/12/02011  24/1966/20/ГПД/130-ДО/20   
888    97436/12/02012  24/1966/20/ГПД/130-ДО/20   
889    97436/12/02013  24/1966/20/ГПД/130-ДО/20   
...               ...                       ...   
22332  97436/12/03212  24/1966/20/ГПД/130-ДО/20   
22333  97436/12/03213  24/1966/20/ГПД/130-ДО/20   
22334  97436/12/03214  24/1966/20/ГПД/130-ДО/20   
22335  97436/12/03215  24/1966/20/ГПД/130-ДО/20   
22336  97436/12/03216  24/1966/20/ГПД/130-ДО/20   

                                 col2           col3  \
885    ООО "Газпром добыча Краснодар"  ГПУ Каневское   
886    ООО "Газпром добыча Краснодар"  ГПУ Каневское   
887    ООО "Газпром добыча Краснодар"  ГПУ Каневское   
888    ООО "Газпром добыча Краснодар"  ГПУ Каневское   
889    ООО "Газпром добыча Краснодар"  ГПУ Каневское   
...                               ...            ..

In [58]:

#for i in ITC_ROS:
df1 = df[df['col31'].isin(ITC_ROS)]
xlsx_path = os.path.join(pathfile,"isp", "ГДДО22 СВОД_АО Газпром диагностика ИТЦ Ростов.xlsx") 
update_spreadsheet(xlsx_path, df1.sample(len(df1)), sheetname="План", starcol=1, startrow=6)

#for i in ITC_SAN:
df2 = df[df['col31'].isin(ITC_SAN)]
xlsx_path = os.path.join(pathfile,"isp", "ГДДО22 СВОД_АО Газпром диагностика ИТЦ Санкт-Петербург.xlsx")  
update_spreadsheet(xlsx_path, df2.sample(len(df2)), sheetname="План", starcol=1, startrow=6)

#for i in ITC_ROS:
df3 = df[df['col31'].isin(ITC_VID)]
xlsx_path = os.path.join(pathfile,"isp", "ГДДО22 СВОД_АО Газпром диагностика ИТЦ Видное.xlsx")
update_spreadsheet(xlsx_path, df3.sample(len(df3)), sheetname="План", starcol=1, startrow=6)

print('ok')
    

ok


In [59]:
po = list(set(isp)- set(ITC))
print(po)
for i in po :    
    xlsx_path = os.path.join(pathfile,"isp", "ГДДО22 СВОД_"+str(i.replace('"',"",2)) +".xlsx")
    df1 = df[df['col31']==i]
    update_spreadsheet(xlsx_path, df1.sample(len(df1)), sheetname="План", starcol=1, startrow=6)
    

['ООО "ГАЗМАШПРОЕКТ"', 'АО "Газприборавтоматикасервис"', 'РГУ нефти и газа (НИУ) имени И.М. Губкина', 'ООО "Газпром недра"', 'ГД-2022_15968', 'ООО "ИФДМ"', 'ООО "Монолит-СПб"', 'ООО "НПЦ ВТД"', 'ООО "НИИПГАЗА"', 'ФГАУ "НУЦСК при МГТУ им. Н.Э. Баумана"', 'ООО "ЭНТЭ"', 'АО НДЦ НПФ "Русская лаборатория"', 'ООО "Газпроект-ДКР"']
